In [ ]:
from sqlalchemy import create_engine, text
from pathlib import Path

import getpass
import sqlalchemy as sa
import os

In [ ]:
pio_repo = Path('/home/jovyan/ohdsi-omop-pioneer/').resolve()

## Settings

Replace these parameters

In [ ]:
CDM_SCHEMA = 'active_biotech'
RESULTS_SCHEMA = CDM_SCHEMA + '_results'

# Connect to the db

In [ ]:
server = 'postgresql'
port = '5432'
db = 'ohdsi'
engine = create_engine(f'postgresql://{input("User:")}:{getpass.getpass("Password:")}@{server}:{port}/{db}')

In [ ]:
sa.inspect(engine).get_schema_names()

In [ ]:
if CDM_SCHEMA in sa.inspect(engine).get_schema_names():
    print('WARNING: the schema already exists. Proceed with care')

## Now  do the following outside this notebook:
- #### Restore the data schema form the dump file (in pgadmin)
- #### Apply the indexes on the CDM clinical tables

## Add vocab views

In [ ]:
con = engine.connect()

In [ ]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
with open(pio_repo.joinpath('other/vocab_view.sql')) as f_vocab_view:
    con.execute(f_vocab_view.read())

## Setup results schema

In [ ]:
con = engine.connect()

In [ ]:
# con.execute('DROP SCHEMA %s CASCADE;' % RESULTS_SCHEMA)
con.execute('CREATE SCHEMA %s;' % RESULTS_SCHEMA)

In [ ]:
con.execute('SET search_path TO %s;' % RESULTS_SCHEMA)
with open(pio_repo.joinpath('other/results_ddl_2.7.4.sql')) as f_results:
    con.execute(text(f_results.read()).execution_options(auto_commit=True))

In [ ]:
con.execute('CREATE OR REPLACE VIEW concept_hierarchy AS (SELECT * FROM vocab.concept_hierarchy);')

Somehow Achilles doesn't create the results_dist table, copy from other results schema

In [ ]:
con.execute('CREATE TABLE %s.achilles_results_dist AS (SELECT * FROM synpuf_1k_results.achilles_results_dist) WITH NO DATA' % RESULTS_SCHEMA)

In [ ]:
con.close()

## Source Daimon

_manual step_

To add in Atlas->Configuration

Note: had to 'Clear Configuration Cash' to make it work. Gettign EntityExsistsErrors before that

## Run Achilles (R)

Run Achilles R script

# End of new data source setup steps

========================================================